# Lab Assignment Two: Exploring Image Data

In [1]:
# Importing Tools
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import re
import plotly
from plotly.graph_objs import Bar, Line
from plotly.graph_objs import Scatter, Layout
from plotly.graph_objs.scatter import Marker
from plotly.graph_objs.layout import XAxis, YAxis
from sklearn.decomposition import PCA
import os, random, shutil
import seaborn as sns

ModuleNotFoundError: No module named 'plotly'

## Business Understanding

Within the production of legos, millions of legos are created on the daily. It is a necessary task to be able to recogonize the characteristics of different bricks and sort them accordingly. In the below dataset, there are 40,000 digitally recreated images of 50 different lego pieces at 800 different angles. This dataset was originally designed for research and learning purposes, and it was designed to train an algorithm to categorize different lego pieces based on appearance. The goal is to be able to predict what kind of lego brick is in a photo with a 98% accuracy. If the model were to achieve any lower levels of accuracy, the amount of displaced legos would be in the hundred thousands daily. Generally, 2% is considered standard error to be accepted. 

**Data Set :**
Images of Lego Brick - *https://www.kaggle.com/datasets/joosthazelzet/lego-brick-images*

## Data Preparation

#### Determine Information about Images being Read In

The information is saved within a single folder full of pngs. The title of each png is it's ID, the name of the piece, and then the size. To gather all the information about the pngs, we must first save all the urls, so that they can all be loaded into our program. Then we need to seperate the name and size from the path, so that we can label each of our images.</br>
Due to restrictions in computing power, and my computer's constant tendancy to run out of RAM to store arrays, I only took the 2667 images. More images than that caused my processor to be unable to complete the necessary calculations. Since there were 800 images for each brick type, taking any consequetive images would cause us to only be able to identify two or three types of bricks. As the shapes are quite simple, it should not take 800 images to train an algorithm to recognize it. Additionally, the recognizition of only a select few types would lend very little help in sorting legos. Resultantly, I chose to take a every 15th image, which allows us to have an even distribution of all the bricks and consistency among the angles. Roughly, we should end up with 53 images per brick</br>
I also chose not to include the ID of the lego brick in the name, as it cluttered our later images and did not carry much necessary meaning. 

In [ ]:
#Referenced Datalira at https://www.kaggle.com/databeru
#Create a list with the filepaths
dir = Path('C:/Users/katie/Documents/GitHub/Data_Science/ML_in_Python/Data/Lego_Images/test/dataset')
    
file_paths = list(dir.glob(r'**/*.png'))

df = [str(x) for x in file_paths[::15]]
df = pd.DataFrame({'Filepath':file_paths[::15]})

#Add Labels from path
def get_label(string):
    #Isolate name from path
    label  = ' '.join(string.split('/')[-1].replace('.png', '').split(' ')[1:-1])
    label = label.lower()
    #remove initial id value
    label = label[re.search(r'[a-z]', label).start():]
    return label
           
# Retrieve the label from the path of the pictures
df['Label'] = df['Filepath'].apply(lambda x: get_label(str(x)))

#### Convert Images to Greyscale and Lower Resolution

To reduce our images from several channels to 1, I chose to read in the images in greyscale. This allowed for each in later linearizing the data into a 1D array, which would have been impossible with more than 1 channels. The CYMK colors did not provide any use in recognizing the features of the bricks, and having the colors would make the process noisier and more difficult</br>
I then chose to remove unnecessary computational time by cropping the images of much of the white space surrounding the borders. Via trial and error, I found that there were roughly 90 additional pixels on every edge of just white space. Because this didn't aid in our feature recognizition and was taking up space that I quite frankly did not have, I simply cropped it out. </br>
As the features on each images are quite large, the details of each image do not need to be very precise to maintain accuracy. So to again save the space needed to process each image, I lowered the quality to roughly 1/2 of it's original resolution. 

In [ ]:
for url in df['Filepath']: 
    #Open file in grayscale
    img = Image.open(url).convert('L')
    #Crop image down
    img = img.crop((90, 90, 310, 310))
    #save at a lower resolution
    img.save(url, quality=1)

#### Read in Images as Numpy Arrays

Now that the images have been converted to a form that my processor can handle, I then converted all of the images into a numpy array for further analysis

In [ ]:
#Convert images to a numpy arrays
original_images = df['Filepath'].apply(lambda x: np.asarray(Image.open(x)))

There are 2,667 images, with a shape of (220, 220). From the shape, we can tell that the photos are all 220x220 pixels. Therefore in our matrix, the rows would represent the number of images, so there are 2667 rows. And the columns are going to be the number of pixels so there are 484000 columns. Because we are in grayscale, each pixel does correlate with only with one column. We can also tell that they all have 1 channels because it does not appear in the shape, which means they range from white at 255 to black at 0. There are 46 unique labels, and therefore 46 types of bricks within this dataset.

In [ ]:
#Print information about dataset
print(len(original_images))
print(original_images[0].shape)
print(len(df['Label'].unique()))

#### Linearize the Images to Create a Table of 1-D Image Features

In order to do further analysis on the images, I flattened my 2D images into 1D arrays. To do this, I first created an array with the number of rows equal to the number of images and the number of columns equal to the number of pixels in each images. I then filled them with zeros as a base. Following this creation, I read in each image, flattened it to 1D, and then stored it in my newly created array. </br>
Then I converted my 2D array of all my 1D images into a dataframe, so that I can continue to run pandas functions on it. 

In [ ]:
#array with there are 2000 rows and 48400 columns
img_data = np.zeros((2667, 48400))

for i in range(len(original_images)):
    img = original_images[i]
    farr = img.flatten()
    img_data[i] = farr

In [ ]:
#Convert array to a dataframe
flat_images = pd.DataFrame(img_data)

flat_images.head()

#### Vizualize Images

Once all my images are loaded and ready to analyze, it was important to view the images and ensure they still looked as I expected them to. As I was going to frequently need to view the images I was analayzing, I utilized Dr. Larson's code to create a gallery of images.</br>
I then printed the first 18 images, in which I saw what I expected. They were black and white images of the lego blocks. All of the block was still shown, and it was still highly recognizable. 

In [ ]:
#Reference from https://github.com/eclarson/MachineLearningNotebooks/blob/master/04.%20Dimension%20Reduction%20and%20Images.ipynb
plt.rcParams['figure.figsize'] = (10.0, 10.0)

#Images : df of images in a numpy array, titles : labels of what each image shows, h : height in pixels, w : width in pixels
#n_rows : number of rows of images (auto 3), n_col : number of images per row (auto 6)
def plot_gallery(images, titles, h, w, n_row=3, n_col=6):
    plt.figure(figsize=(1.7 * n_col, 2.3 * n_row))
    plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
    for i in range(n_row * n_col):
        plt.subplot(n_row, n_col, i+1)
        try:
            plt.imshow(images.iloc[i].values.reshape(h,w), cmap=plt.cm.gray)
        except:
            plt.imshow(images[i].reshape(h,w), cmap=plt.cm.gray)
        plt.title(titles[i], size=12)
        plt.xticks(())
        plt.yticks(())
        
    plt.show()

In [ ]:
plot_gallery(flat_images, df['Label'], 220, 220)

## Data Reduction

#### Linear dimensionality reduction --- Full PCA

In order to derive new features from the images we have, we need to use dimensional analysis. Our first method is a full Principal Component Analysis, which helps us capture the largest amount of variation in data. In order to understand the information we are going to work with, we need to gather the samples, features, classes, and size of the data. The samples is the number of images we have to analyze. The features is the number of different kinds of bricks there are. The classes are the number of labels. h is the height of the images in pixels, and w is the width of images in pixels. </br>
We also printed out a sample of the images, just as a reference of what we're looking at.

In [ ]:
X = flat_images
y = df['Label']

target_names = df['Label']

n_samples, n_features = X.shape
h, w = original_images[0].shape
n_classes = len(target_names)

print("n_samples: {}".format(n_samples))
print("n_features: {}".format(n_features))
print("n_classes: {}".format(n_classes))
print("Original Image Sizes {} by {}".format(h,w))
#Print Information about data

In [ ]:
plot_gallery(X, target_names, h, w)

Now that we have our information, we can move forward with calculating our PCA. We pull out all the components, which we choose to be the number of samples we have because are doing a complete analysis. The PCA is calculated for us by finding the eigenvectors and then the covariance matrix. 

In [ ]:
#Full PCA 
n_components = 2667
print ("Extracting the top %d eigenpieces from %d lego faces" % (n_components, X.shape[0]))

pca = PCA(n_components=n_components)
pca.fit(X.copy())
eigenpieces = pca.components_.reshape((n_components, h, w))

Each principlate component shows you a certain amount of variation in the data which is encoded in the eigenvalue for each eigenvector. It is found through finding the factors of the eigen values. It can be calculated via the formula below.</br>
The plot that is formed shows that most of the variance can be explained by the principal component at around 30% variance. The 2-5 components also adds roughly about 30% of the variance. From there the remaining components, the remaining 40% are accumulated, and they trail off exponentially. When interacting with below plot, we can see that we reach 90% explained variance with roughly 100 components. 100 out of over 2000 components is very low, which means we can reach a very high fidelity reconstruction with very few components</br>
Ultimately, we have determined that we can gather a more clear reconstruction with our full dataset, even though the first couple images covers most of our data. 

$$ r_q=\frac{\sum_{j=1}^q \lambda_j}{\sum_{\forall i} \lambda_i} $$

In [ ]:
#Calculate Explained Variance for Each Component
#Reference from https://github.com/eclarson/MachineLearningNotebooks/blob/master/04.%20Dimension%20Reduction%20and%20Images.ipynb
def plot_explained_variance(pca):
    plotly.offline.init_notebook_mode() 
    
    explained_var = pca.explained_variance_ratio_
    cum_var_exp = np.cumsum(explained_var)
    
    plotly.offline.iplot({
        "data": [Bar(y=explained_var, name='individual explained variance'),
                 Scatter(y=cum_var_exp, name='cumulative explained variance')
            ],
        "layout": Layout(xaxis=XAxis(title='Principal components'), yaxis=YAxis(title='Explained variance ratio'))
    })

In [ ]:
plot_explained_variance(pca)

Once the eigenvalues of images are calculated, we can visualize them to see the reduction analysis. They maintain the same general features without the clutter. They are extremely hard to distinguish to the human eye, but that just falls into the nature of the data.Each of the pieces represents a feature of all images rather than specific to one image.

In [ ]:
eigenpiece_titles = ["eigenpiece %d" % i for i in range(eigenpieces.shape[0])]
plot_gallery(eigenpieces, eigenpiece_titles,h,w)

#### Linear dimensionality reduction --- Randomized PCA

Utilizing the same methodology of full PCA, we can create a random PCA using 2667 randomized components. Again, the resulting images are very vague due to the nature of the data, but the features are still emphasized. 

In [ ]:
# lets do some PCA of the features and go from 2667 features to 5 features
n_components = 2667
print ("Extracting the top %d eigenpieces from %d lego pieces" % (
    n_components, X.shape[0]))

rpca = PCA(n_components=n_components, svd_solver='randomized')
%time rpca.fit(X.copy())
eigenpieces = rpca.components_.reshape((n_components, h, w))

In [ ]:
eigenpiece_titles = ["eigenpiece %d" % i for i in range(eigenpieces.shape[0])]
plot_gallery(eigenpieces, eigenpiece_titles,h,w,1,5)

The plot that is formed shows that most of the variance can be explained by the principal component at around 30% variance. The 2-5 components also adds roughly about 30% of the variance. From there the remaining components, the remaining 35% are accumulated, and they trail off exponentially. Therefore, it adheres to the same pattern as before.</br>
We reach an exponential variance level of 85%, which is slightly below what we developed above. Again, the most ideal level of 90% occurs at roughly 100 principal components.

In [ ]:
plot_explained_variance(rpca)

#### Full vs Randomized PCA

We now want to compare full PCA and Randomized PCA. Below I reconstructed an image utilizing each of the two methods. We can see that the full PCA creates a sharper images than the randomized PCA. Most likely, this is due to the full PCA having more consequtive components to create features from. However, I did find that it was more convenient to use the randomized PCA because the method was able to run much more efficiently due to the lower number of calculations needed. We can see from below, the image is still recognizable without sacrificing efficiency. However, ultimately both plots appear to require the same number of components to produce a sufficiently accurate model, so they are equally adequate for this particular run. 

In [ ]:
#Images Reconstruction of two types to compare
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.axis('off')

for index_to_reconstruct in [10, 50, 80, 100] :
    
    plt.figure(figsize=(15,7))
    
    #Original
    des = original_images[index_to_reconstruct].reshape(1, -1)
    plt.subplot(1,4,1)
    plt.imshow(original_images[index_to_reconstruct].reshape((h, w)), cmap=plt.cm.gray)
    plt.title('Original')
    plt.grid()

    #Full PCA
    reconstructed_image = pca.inverse_transform(pca.transform(des))
    plt.subplot(1,4,2)
    plt.imshow(reconstructed_image.reshape((h, w)), cmap=plt.cm.gray)
    plt.title('Full PCA')
    plt.grid()

    #Randomized PCA
    reconstructed_image_rpca = rpca.inverse_transform(rpca.transform(des))
    plt.subplot(1,4,3)
    plt.imshow(reconstructed_image_rpca.reshape((h, w)), cmap=plt.cm.gray)
    plt.title('Randomized PCA')
    plt.grid()
    
plt.show()

## Feature Extraction

#### Daisy Feature Extraction

In order to perform a feature extraction, I chose to use DAISY. DAISY makes the image convolved with Gausian kernel function. It then calculates the projection of the gradient of each pixel by making each feature a vector with each element being the gradient. To test, I chose a step of 20 and a radius of 20 to create overlap so the feature extraction would get a better . Below, we were able to extract 5580 features from our test image. 

In [ ]:
from skimage.feature import daisy

idx_to_reconstruct = 21
img  = original_images[idx_to_reconstruct].reshape((h,w))
features, img_desc = daisy(img, step=20, radius=20, rings=2, histograms=8, orientations=8, visualize=True)
plt.imshow(img_desc)
plt.grid(False)

In [ ]:
features = daisy(img, step=20, radius=20, rings=2, histograms=8, orientations=4, visualize=False)
print(features.shape)
print(features.shape[0]*features.shape[1]*features.shape[2])

I then took the above feature test and applied to to all the images in my 1D array of images. We were able to determine 5508 features from this analysis.

In [ ]:
# create a function to tak in the row of the matric and return a new feature
def apply_daisy(row,shape):
    feat = daisy(row.reshape(shape), step=20, radius=20, 
                 rings=2, histograms=8, orientations=4, 
                 visualize=False)
    return feat.reshape((-1))

%time test_feature = apply_daisy(original_images[idx_to_reconstruct],(h,w))
test_feature.shape

In [ ]:
# apply to entire data, row by row
daisy_features = np.apply_along_axis(apply_daisy, 1, X, (h,w))
print(daisy_features.shape)

#### Nearest Neighbor Analysis

Now that we have completed said feature analysis, we must now determine it's effectiveness. The first step was to see if it could determine what image was closest to a given image. From the below code, I was able to display the original image and what Daisy determined to be the closest image. As we can see, it gave us an image of a completely other brick, which would cause incorrect classification. Because of this, the DAISY method does not seem entirely helpful.

In [ ]:
import copy
# find closest image to current image
idx1 = 5
distances = copy.deepcopy(dist_matrix[idx1,:])
distances[idx1] = np.infty # dont pick the same image!
idx2 = np.argmin(distances)

plt.figure(figsize=(7,10))
plt.subplot(1,2,1)
plt.imshow(original_images[idx1].reshape((h,w)))
plt.title("Original Image")
plt.grid()

plt.subplot(1,2,2)
plt.imshow(original_images[idx2].reshape((h,w)))
plt.title("Closest Image")
plt.grid()

#### Heatmap of Pairwise Differences

We first created a heatmap based on the pairwise distances of the daisy features, but there was too much data and the information was impossible to read. 

In [ ]:
dist_matrix_daisy = pairwise_distances(daisy_features)

plt.rcParams['figure.figsize'] = (20.0, 20.0)
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(dist_matrix_daisy, cmap=cmap)
plt.show()

To make the heatmap more understandable, we first break everything up into it's categories, and then apply the daisy algorithm from there. We take the mean value of each column of each matrix of bricks, and then drawing the pair-wise difference.

In [ ]:
idic = {}

for i in range(len(X)):
    ci = df['Label'][i]
    tmparray = []
    li = idic.get(ci, tmparray)
    tmprow = X.iloc[i]
    li.append(tmprow)
    idic[ci] = li
print(idic.keys())

In [ ]:
tmparr = []

for i in idic.keys():
    cur_feature = np.mean(np.apply_along_axis(apply_daisy, 1, idic.get(i), (220,220)), axis=0).reshape((1,5508)).ravel()
    tmparr.append(cur_feature)
tmparr = np.asarray(tmparr)
tmparr.shape

In [ ]:
dist_matrix_daisy = pairwise_distances(tmparr)
dist_matrix_daisy.shape

The resultant plot below shows that there is not big different from one brick to another. That means that there are varying differences with no particular pattern. If the block is red than there is some notable differnce in that type of brick. Therefore, there is some information in the pixel that we can extract for images. Therefore, there is a chance that we could extact the categories, but due to the lack of notability in most of the bricks, it does not seem likely the feature detection via DAISY will be accurate enough to serve our purpose.

In [ ]:
plt.rcParams['figure.figsize'] = (20.0, 20.0)
#plt.axis('off')

cmap = sns.diverging_palette(220, 10, as_cmap=True)
#sns.set(style="darkgrid")
#f, ax = plt.subplots(figsize=(50, 50))
sns.heatmap(dist_matrix_daisy, cmap=cmap)
#f.tight_layout()
plt.show()